$\qquad$ $\qquad$$\qquad$  **TDA 232 Machine Learning: Home Assignment 3 -- Classification** <br />
$\qquad$ $\qquad$$\qquad$ **Logistic Regression (Y)**<br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Yuchong (Y), Divya (D)** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 6th May** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Yaxi Xie, 9604039207, yaxi@student.chalmers.se** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by: Karl-Rehan Chiu Falck, 9411237499, falckk@student.chalmers.se** <br />

---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   **All the answers for theoretical questions must be filled in the cells created for you with "Your answer here" below each question, but feel free to add more cells if needed.**
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Practical Question
## Logistic Regression (5 pts)
### Customer churn with Logistic Regression
A telecommunications company is concerned about the number of customers leaving their land-line business for cable competitors. They need to understand who is leaving. Imagine that you are an analyst at this company and you have to find out who is leaving and why.

### About the dataset

We will use a telecommunications dataset for predicting customer churn. This is a historical customer dataset where each row represents one customer. The data is relatively easy to understand, and you may uncover insights you can use immediately. Typically it is less expensive to keep customers than acquire new ones, so the focus of this analysis is to predict the customers who will stay with the company.
This data set provides information to help you predict what behavior will help you to retain customers. You can analyse all relevant customer data and develop focused customer retention programs.
The dataset includes information about:
*   Customers who left within the last month – the column is called Churn.
*   Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies.
*   Customer account information – how long they had been a customer, contract, payment method, paperless billing, monthly charges, and total charges.
*   Demographic info about customers – gender, age range, and if they have partners and dependents.
We will help you load and visualise the dataset as well as the preprocessing, you need to build up your logistic regression model step by step and do the prediction.
*   **Remember, you are not allowed to use sklearn in modelling and predicting, you have to fill your code in the skeleton.** 








In [1]:
## Load the dataset and read it
import pandas as pd
import pylab as pl
import numpy as np
import scipy.optimize as opt
from sklearn import preprocessing
import urllib

try:
    urllib.request.urlretrieve('https://raw.githubusercontent.com/BruceZHANG6/Imagesforuse/master/ChurnData.csv', 'ChurnData.csv')
except urllib.error.HTTPError as ex:
    print('Problem:', ex)
    
churn_df = pd.read_csv("ChurnData.csv")
churn_df.head()

,tenure,age,address,income,ed,employ,equip,callcard,wireless,longmon,...,pager,internet,callwait,confer,ebill,loglong,logtoll,lninc,custcat,churn
0,11.0,33.0,7.0,136.0,5.0,5.0,0.0,1.0,1.0,4.40,...,1.0,0.0,1.0,1.0,0.0,1.482,3.033,4.913,4.0,1.0
1,33.0,33.0,12.0,33.0,2.0,0.0,0.0,0.0,0.0,9.45,...,0.0,0.0,0.0,0.0,0.0,2.246,3.240,3.497,1.0,1.0
2,23.0,30.0,9.0,30.0,1.0,2.0,0.0,0.0,0.0,6.30,...,0.0,0.0,0.0,1.0,0.0,1.841,3.240,3.401,3.0,0.0
3,38.0,35.0,5.0,76.0,2.0,10.0,1.0,1.0,1.0,6.05,...,1.0,1.0,1.0,1.0,1.0,1.800,3.807,4.331,4.0,0.0
4,7.0,35.0,14.0,80.0,2.0,15.0,0.0,1.0,0.0,7.10,...,0.0,0.0,1.0,1.0,0.0,1.960,3.091,4.382,3.0,0.0


In [2]:
## Data pre-processing and selection
## Train/Test dataset split
churn_df = churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip',   'callcard', 'wireless','churn']]
churn_df['churn'] = churn_df['churn'].astype('int')
churn_df.head()

X = np.asarray(churn_df[['tenure', 'age', 'address', 'income', 'ed', 'employ', 'equip']])
y = np.asarray(churn_df['churn'])
y = np.reshape(y, (np.asarray(churn_df['churn']).shape[0], 1))
X = preprocessing.StandardScaler().fit(X).transform(X)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
X_train=X_train.T
X_test=X_test.T
y_train=y_train.T
y_test=y_test.T
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (7, 160) (1, 160)
Test set: (7, 40) (1, 40)


**Hints**:
- You compute $sigmoid( w^T x + b) = \frac{1}{1 + e^{-(w^T x + b)}}$ .
- You compute activation $A = \sigma(w^T X + b) = (a^{(0)}, a^{(1)}, ..., a^{(m-1)}, a^{(m)})$.
- You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$.

Here are the two formulas you will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})$$

- You write down the optimization function. The goal is to learn $w$ and $b$ by minimizing the cost function $J$. For a parameter $\theta$, the update rule is $ \theta = \theta - \alpha \text{ } d\theta$, where $\alpha$ is the learning rate.
- In prediction, you calculate $\hat{Y} = A = \sigma(w^T X + b)$.
- You may use np.exp, np.log(), np.dot(), etc. 

In [3]:
## Modeling and predicting

# GRADED FUNCTION: sigmoid
def sigmoid(z):

#    Return: s -- sigmoid(z)

    s = 1.0/(1+np.exp(-z))
    
    return s
  
# GRADED FUNCTION: initialize_with_zeros
def initialize_with_zeros(dim):
#    """
#    Returns:
#    w -- initialized vector of shape (dim, 1)
#    b -- initialized to 0
#    """    
    w = np.empty(shape=[dim, 1])#np.zeros((dim,1))
    b = np.zeros((1,1))
   
    assert(w.shape == (dim,1))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b
  
# GRADED FUNCTION: grad_cost
def grad_cost(w, b, X, Y):
 # """
 #   Arguments:
 #   X -- data of size (number of features, number of examples)
 #   Y -- true "label" vector

 #   Return:
 #   cost -- negative log-likelihood cost for logistic regression
 #   dw -- gradient of the loss with respect to w, thus same shape as w
 #   db -- gradient of the loss with respect to b, thus same shape as b
 # """ 
    m = X.shape[1]
    A = sigmoid((np.dot(w.T, X) + b))                                  
    cost = -(np.dot(np.log(A),Y.T)+np.dot(np.log(1-A),(1-Y).T))/m

    dw = np.dot(X,(A-Y).T)/ m 
    db = np.sum((A-Y), 1)/m
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

 # GRADED FUNCTION: optimize
def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
 #   """
 #   This function optimizes w and b by running a gradient descent algorithm
 #   """
    costs = []
    
    for i in range(num_iterations):
        
        grads, cost = grad_cost(w,b,X,Y)
        dw = grads["dw"]
        db = grads["db"]
        
        # update rule
        w -= np.dot(dw,learning_rate)
        b -= db*learning_rate
        
        if i % 100 == 0:
            costs.append(cost)     
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}   
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs
  
# GRADED FUNCTION: predict
def predict(w, b, X):
 #   '''
 #   Returns:
 #   Y_prediction -- a numpy array (vector) containing all predictions (0/1) for the examples in X
 #   '''    
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    b = b.reshape(1, 1)
    
    # Compute vector "A" predicting the probabilities of a cat being present in the picture
    A = sigmoid((np.dot(w.T, X) + b))  
    
    for i in range(A.shape[1]):
        
        if A[0,i]<=0.5:
            Y_prediction[0,i]=0;
        if A[0,i]>0.5:
            Y_prediction[0,i]=1
    
    assert(Y_prediction.shape == (1, m))
    
    return Y_prediction
  
# GRADED FUNCTION: model
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
 #   """
 #   Builds the logistic regression model by calling the function you've implemented previously
 #   Returns: d -- dictionary containing information about the model.
 #   """    
    # initialize parameters with zeros
    w = np.zeros((X_train.shape[0],1))
    b = np.zeros((1,1))

    # Gradient descent
    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost = False)
    
    w = parameters["w"]
    b = parameters["b"]

    Y_prediction_test = predict(w,b,X_test)
    Y_prediction_train = predict(w,b,X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
   
    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [4]:
## The train accuracy and test accuracy
## Feel free to change the hyperparameters
d = model(X_train, y_train, X_test, y_test, num_iterations = 20000, learning_rate = 0.005, print_cost = True)

train accuracy: 77.5 %
test accuracy: 72.5 %
